# Term project : Milestone 4
API Data pull
### Yograj Karki

For this project, I'm going to pull historical weather data from Source: https://openweathermap.org/api

In [53]:
# Loading libraries
import pandas as pd
import requests
import json

In [54]:
# API key
api_key= "ec160a45a9812a127cc354d8a5adea88"


In [55]:
# Importing sampled accidents data file
df = pd.read_csv("sample_accidents.csv")
df.shape

(145000, 47)

In [56]:
df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-30,2,2019-11-25 07:11:55,2019-11-25 08:42:16,43.220390,-85.500961,43.220390,-85.500961,0.0,Accident on 17 Mile Rd at Stout Ave.,...,False,False,False,False,False,False,Night,Night,Day,Day
1,A-44,2,2016-12-29 07:35:34,2016-12-29 08:05:17,28.698406,-82.451477,28.698406,-82.451477,0.0,Accident on County Hwy-480 Oak Park Blvd at Co...,...,False,False,False,False,False,False,Day,Day,Day,Day
2,A-67,2,2019-04-11 12:20:07,2019-04-11 12:49:16,31.759741,-106.484001,31.759741,-106.484001,0.0,Accident on Campbell St at Myrtle Ave.,...,False,False,False,False,True,False,Day,Day,Day,Day
3,A-107,2,2017-08-22 10:53:29,2017-08-22 11:23:16,41.256569,-95.940956,41.256569,-95.940956,0.0,Accident on 19th St Eastbound at Harney St.,...,False,True,False,False,True,False,Day,Day,Day,Day
4,A-123,2,2019-06-19 17:55:06,2019-06-19 18:24:54,33.713829,-112.285217,33.713829,-112.285217,0.0,Accident on 107th Ave at Happy Valley Pkwy.,...,False,False,False,False,True,False,Day,Day,Day,Day


In [57]:
# creating a new data frame for weather data
df2 = df[["ID","Start_Time","City"]].copy()
df2.head()

,ID,Start_Time,City
0,A-30,2019-11-25 07:11:55,Cedar Springs
1,A-44,2016-12-29 07:35:34,Homosassa
2,A-67,2019-04-11 12:20:07,El Paso
3,A-107,2017-08-22 10:53:29,Omaha
4,A-123,2019-06-19 17:55:06,Peoria


In [58]:
import time
import datetime


In [59]:
#df2 = pd.to_datetime(df["Start_Time"])
df2['timestamp'] = df2['Start_Time'].apply(lambda x: pd.Timestamp(x))


In [60]:
df2

,ID,Start_Time,City,timestamp
0,A-30,2019-11-25 07:11:55,Cedar Springs,2019-11-25 07:11:55
1,A-44,2016-12-29 07:35:34,Homosassa,2016-12-29 07:35:34
2,A-67,2019-04-11 12:20:07,El Paso,2019-04-11 12:20:07
3,A-107,2017-08-22 10:53:29,Omaha,2017-08-22 10:53:29
4,A-123,2019-06-19 17:55:06,Peoria,2019-06-19 17:55:06
...,...,...,...,...
144995,A-2906521,2020-12-23 22:13:00,Williamsport,2020-12-23 22:13:00
144996,A-2906544,2019-12-24 15:51:11,El Paso,2019-12-24 15:51:11
144997,A-2906551,2019-07-06 13:02:31,Portland,2019-07-06 13:02:31
144998,A-2906562,2020-03-28 11:42:00,Madera,2020-03-28 11:42:00


In [19]:
s = "2019-06-19 17:55:06"
time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple())


df['Discounted_Price'] = df.apply(lambda row: row.Cost - 
                                  (row.Cost * 0.1), axis = 1)

1560992106.0

In [21]:
df["timestamp"] = df.apply(lambda row: row.Start_Timetime.mktime(datetime.datetime.strptime(df["Start_Time"], "%Y-%m-%d %H:%M:%S").timetuple())

TypeError: strptime() argument 1 must be str, not Series

In [ ]:
# Creating a function to retrieve weather data by City name and date
def 

In [45]:
city = "Boston"
country_code = "US"
start_time = 1560992106.0

url= "http://history.openweathermap.org/data/2.5/history/city?q={},{}&type=hour&start={}&appid={}".format(city,country_code, start_time, api_key)

data = requests.get(url).json()
#data['main']['humidity']

In [46]:
data

{'cod': 401,
 'message': 'Invalid API key. Please see http://openweathermap.org/faq#error401 for more info.'}

In [50]:
url = "http://api.openweathermap.org/data/2.5/forecast?id=524901&appid=ec160a45a9812a127cc354d8a5adea88".format(api_key)
data = requests.get(url).json()
data

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1627884000,
   'main': {'temp': 293.32,
    'feels_like': 293.48,
    'temp_min': 293.32,
    'temp_max': 296.07,
    'pressure': 1010,
    'sea_level': 1010,
    'grnd_level': 992,
    'humidity': 80,
    'temp_kf': -2.75},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': {'all': 100},
   'wind': {'speed': 2.91, 'deg': 147, 'gust': 4.53},
   'visibility': 10000,
   'pop': 0.5,
   'rain': {'3h': 0.22},
   'sys': {'pod': 'd'},
   'dt_txt': '2021-08-02 06:00:00'},
  {'dt': 1627894800,
   'main': {'temp': 297.33,
    'feels_like': 297.45,
    'temp_min': 297.33,
    'temp_max': 300.02,
    'pressure': 1009,
    'sea_level': 1009,
    'grnd_level': 991,
    'humidity': 63,
    'temp_kf': -2.69},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04d'}],
   'clouds': {'all': 97},
   'wind': {'speed': 5.11, 'deg': 1

In [35]:
data

{'cod': 401,
 'message': 'Invalid API key. Please see http://openweathermap.org/faq#error401 for more info.'}

In [27]:
df.columns

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [25]:
url = "http://history.openweathermap.org/data/2.5/history/city?id={}&type=hour&appid={}""

SyntaxError: invalid syntax (<ipython-input-25-b1c20ae1c3fa>, line 1)